In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

TRAIN_DATASET_PATH = "../datasets/meds_train_alunos.csv"
TEST_DATASET_PATH = "../datasets/meds_test_alunos.csv"
BTS_DATASET = "../datasets/Bts.csv"
IMG_PATH = "imgs/"

In [43]:
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_train.head()

,ponto_id,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,lat,lon,indoor
0,6604,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,5.0,3.0,2.0,-8.055786,-34.951505,True
1,5515,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,5.0,3.0,2.0,-8.055784,-34.951608,True
2,5049,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,5.0,3.0,2.0,-8.055506,-34.951693,True
3,8737,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,4.0,7.0,2.0,-8.050390,-34.959630,False
4,1203,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,5.0,3.0,2.0,-8.055380,-34.951736,True


In [44]:
df_train_noID = df_train.drop(['ponto_id'],axis=1)
df_train_noDuplicatas = df_train_noID.drop_duplicates()

In [45]:
df_train_noDuplicatas.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,lat,lon,indoor
0,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,5.0,3.0,2.0,-8.055786,-34.951505,True
1,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,5.0,3.0,2.0,-8.055784,-34.951608,True
2,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,5.0,3.0,2.0,-8.055506,-34.951693,True
3,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,4.0,7.0,2.0,-8.050390,-34.959630,False
4,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,5.0,3.0,2.0,-8.055380,-34.951736,True


In [46]:
def delay_distance(pd):
    return pd*550, (pd+1)*550

def delays_columns(row):
    return delay_distance(row['delay_1'])[0],delay_distance(row['delay_1'])[1], delay_distance(row['delay_2'])[0],delay_distance(row['delay_2'])[1], delay_distance(row['delay_3'])[0], delay_distance(row['delay_3'])[1]

In [47]:
df_train_noDuplicatas[['dist_min_1', 'dist_max_1','dist_min_2', 'dist_max_2','dist_min_3', 'dist_max_3']] = df_train_noDuplicatas.apply(lambda row: delays_columns(row), axis=1, result_type='expand')
df_train_dists = df_train_noDuplicatas.drop(['delay_1','delay_2','delay_3'],axis=1)

C:\Users\JoaoFelipeMoraisVasc\AppData\Local\Temp\ipykernel_23460\1739117259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_noDuplicatas[['dist_min_1', 'dist_max_1','dist_min_2', 'dist_max_2','dist_min_3', 'dist_max_3']] = df_train_noDuplicatas.apply(lambda row: delays_columns(row), axis=1, result_type='expand')
C:\Users\JoaoFelipeMoraisVasc\AppData\Local\Temp\ipykernel_23460\1739117259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_noDuplicatas[['dist_min_1', 'dist_max_1','dist_m

In [49]:
df_train_dists.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,lat,lon,indoor,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
0,-103.705000,-110.425000,-107.381667,-110.330000,-108.345000,-106.590000,-107.153333,-102.120000,-115.00,-8.055786,-34.951505,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
1,-105.225000,-115.000000,-105.210000,-115.000000,-105.835000,-115.000000,-88.470000,-94.024286,-108.12,-8.055784,-34.951608,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
2,-102.610000,-115.000000,-101.380000,-115.000000,-104.295000,-97.360000,-96.980000,-99.572500,-115.00,-8.055506,-34.951693,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0
3,-89.140000,-97.251429,-86.642857,-96.036667,-96.592500,-92.460000,-86.813333,-102.688000,-87.27,-8.050390,-34.959630,False,2200.0,2750.0,3850.0,4400.0,1100.0,1650.0
4,-109.088333,-107.200000,-107.010000,-108.430000,-104.845714,-101.435714,-103.065000,-102.110000,-115.00,-8.055380,-34.951736,True,2750.0,3300.0,1650.0,2200.0,1100.0,1650.0


In [12]:
num_duplicatas = df_train.duplicated().sum()

print("Número de duplicatas no dataset:", num_duplicatas)

Número de duplicatas no dataset: 3369


In [128]:
from sklearn.model_selection import train_test_split
X = df_train_dists.drop(columns=['lat','lon'])
X['indoor'] = X['indoor'].astype(int)
y = df_train_dists.iloc[:,[9,10]]
# Supondo que X seja seu conjunto de features e y seja seu conjunto de labels/targets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.head()


,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,indoor,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
6464,-115.000000,-115.000000,-115.000000,-115.000000,-96.3300,-92.856667,-83.967500,-78.261429,-115.000000,1,2750.0,3300.0,1650.0,2200.0,1650.0,2200.0
4426,-88.008571,-92.167143,-91.385714,-70.280000,-85.0325,-81.864286,-89.371667,-91.868000,-97.600000,0,2200.0,2750.0,550.0,1100.0,2200.0,2750.0
4844,-78.080000,-96.850000,-85.342381,-101.535000,-99.1800,-87.030000,-80.617222,-99.934667,-97.705000,0,1650.0,2200.0,2200.0,2750.0,1100.0,1650.0
2042,-88.720000,-92.850000,-91.991429,-50.590000,-80.3375,-76.121429,-90.261667,-97.146000,-97.600000,0,2200.0,2750.0,550.0,1100.0,2750.0,3300.0
256,-92.876190,-98.737143,-89.902857,-103.375833,-101.5175,-95.780000,-78.347667,-81.350571,-91.538333,0,2750.0,3300.0,2750.0,3300.0,0.0,550.0


In [129]:
from sklearn.preprocessing import StandardScaler

col_to_move = X_train.iloc[:, 9]  # Extrai a coluna que deseja mover
X_train = X_train.drop(X_train.columns[9], axis=1)  # Remove a coluna do DataFrame

# Adiciona a coluna de volta ao DataFrame no final


sc = StandardScaler()
X_train_names = X_train.columns
X_std_train = sc.fit_transform(X_train)

X_std_train = pd.DataFrame(X_std_train, columns=X_train_names)
X_std_train['indoor'] = col_to_move  

In [130]:
X_train.head()

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3
6464,-115.000000,-115.000000,-115.000000,-115.000000,-96.3300,-92.856667,-83.967500,-78.261429,-115.000000,2750.0,3300.0,1650.0,2200.0,1650.0,2200.0
4426,-88.008571,-92.167143,-91.385714,-70.280000,-85.0325,-81.864286,-89.371667,-91.868000,-97.600000,2200.0,2750.0,550.0,1100.0,2200.0,2750.0
4844,-78.080000,-96.850000,-85.342381,-101.535000,-99.1800,-87.030000,-80.617222,-99.934667,-97.705000,1650.0,2200.0,2200.0,2750.0,1100.0,1650.0
2042,-88.720000,-92.850000,-91.991429,-50.590000,-80.3375,-76.121429,-90.261667,-97.146000,-97.600000,2200.0,2750.0,550.0,1100.0,2750.0,3300.0
256,-92.876190,-98.737143,-89.902857,-103.375833,-101.5175,-95.780000,-78.347667,-81.350571,-91.538333,2750.0,3300.0,2750.0,3300.0,0.0,550.0


In [133]:

X_std_train.iloc[1:10]

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,dist_min_1,dist_max_1,dist_min_2,dist_max_2,dist_min_3,dist_max_3,indoor
1,0.676725,0.658911,0.334570,1.754076,0.977188,1.229453,0.217618,0.094873,0.269050,-0.293203,-0.293203,-1.651311,-1.651311,0.873487,0.873487,1.0
2,1.682842,0.207847,0.963723,-0.391529,-0.663219,0.640751,1.255345,-0.807292,0.260394,-1.121566,-1.121566,0.494245,0.494245,-0.508294,-0.508294,1.0
3,0.604632,0.593137,0.271511,3.105763,1.521575,1.883929,0.112120,-0.495411,0.269050,-0.293203,-0.293203,-1.651311,-1.651311,1.564377,1.564377,0.0
4,0.183462,0.026072,0.488946,-0.517899,-0.934253,-0.356429,1.524372,1.271128,0.768765,0.535160,0.535160,1.209430,1.209430,-1.890074,-1.890074,1.0
5,-1.459405,-0.789091,-1.292028,-0.864860,-1.375983,-1.025720,-1.405552,-1.050579,-1.165381,0.535160,0.535160,-0.220940,-0.220940,-0.508294,-0.508294,0.0
6,0.549187,0.726750,0.244220,0.221730,0.292502,0.756731,0.491854,1.065409,0.325109,-1.121566,-1.121566,-0.220940,-0.220940,2.946157,2.946157,0.0
7,-1.067912,-1.540407,-1.104635,-1.315880,-1.434869,-2.546806,-0.127721,-0.146283,-0.598203,0.535160,0.535160,-0.220940,-0.220940,-0.508294,-0.508294,1.0
8,0.565111,0.253806,1.739568,-0.324940,-0.652204,-0.341288,3.249603,0.771497,1.888144,-0.293203,-0.293203,1.209430,1.209430,-1.199184,-1.199184,1.0
9,0.292373,0.574423,0.283855,0.005945,0.119446,-0.209416,-0.083862,0.792299,0.412700,2.191887,2.191887,-0.220940,-0.220940,0.873487,0.873487,1.0
